In [0]:
%pip install google-search-results
%pip install wikipedia

In [0]:
import os
import openai
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, load_tools

In [0]:
openai.api_type = "azure" 
openai.api_base =  "https://kf-llm.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_KEY"] = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["SERPAPI_API_KEY"] = "4ea6865054c8eb8608723170dd33e12314d8e182c10c16ee15565ccf03526e8e"

In [0]:
llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
)

In [0]:
#Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.
tools = load_tools(["serpapi", "llm-math","wikipedia","terminal"], llm=llm) #The tools the agent will have access to are: [Google search, math, wikipedia, terminal]

In [0]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)
tools[1].name, tools[1].description

In [0]:
agent.agent.llm_chain.prompt.template

In [0]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, load_tools
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [0]:
search = SerpAPIWrapper()
tools = [
   Tool(
       name="Search",
       func=search.run,
       description="useful for when you need to answer questions about current events",
   )
]

In [0]:
prefix = """Answer the following questions as best you can, but do it in old Shakepearean English. You have access to the following tools:"""
suffix = """Begin! Remember to speak in old Shakepearean English in the final answer. Use the word "behold" at least once.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
   tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)



print(prompt.template)

In [0]:
llm_chain = LLMChain(llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
), prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

agent_executor.run("How many hurricanes are expected to make landfall in the US this year?")

In [0]:
agent_executor.run("What do we currently know about aliens?")

In [0]:
agent_output_UFO_expert = agent_executor.run("Who is the leading expert on UFOs?")

In [0]:
agent_executor.run("Take the year" + agent_output_UFO_expert + "died and add 100 years. Will this be the year we prove aliens exist?")